In [ ]:
!pip install -q transformers peft accelerate bitsandbytes datasets pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install -q pypdf
from pypdf import PdfReader
reader = PdfReader("/content/drive/MyDrive/NIST.CSWP.29.pdf")
raw_text = "\n".join([page.extract_text() for page in reader.pages])
print(raw_text)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 4.6 MB/s eta 0:00:00
National Institute of Standards and Technology 
This publication is available free of charge from: https://doi.org/10.6028/NIST.CSWP.29 
February 26, 2024  
The NIST Cybersecurity 
Framework (CSF) 2.0 

T he NIST Cybersecurity Framework (CSF) 2.0 
 i 
NIST CSWP 29 
February 26, 2024 
Abstract
T
he NIST Cybersecurity Framework (CSF) 2.0 provides guidance to industry, government 
agencies, and other organizations to manage cybersecurity risks. It offers a taxonomy of high-
level cybersecurity outcomes that can be used by any organization — regardless of its size, 
sector, or maturity — to better understand, assess, prioritize, and communicate its 
cybersecurity efforts. The CSF does not prescribe how outcomes should be achieved. Rather, it 
links to online resources that provide additional guidance on practices and controls that could 
be used to achieve those outcomes. This document describes CSF 2.0, its co

In [ ]:
# STEP 2: Parse PDF & Clean Paragraphs (fallback)
from pypdf import PdfReader
reader = PdfReader("/content/drive/MyDrive/NIST.CSWP.29.pdf")
raw_text = "\n".join([page.extract_text() for page in reader.pages])
paragraphs = [p.strip() for p in raw_text.split("\n")]
print(f" Extracted {len(paragraphs)} paragraphs.")

# STEP 3: Chunk PDF using Langchain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings


loader = PyPDFLoader("/content/drive/MyDrive/NIST.CSWP.29.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ".", " ", ""])
rec_chunks = splitter.split_documents(docs)

# Then semantic chunking on top of recursive result
hf_embed = HuggingFaceEmbeddings(model_name="fdtn-ai/Foundation-Sec-8B")

semantic_splitter = SemanticChunker(
    embedding_model=hf_embed,
    chunk_size=800,
    chunk_overlap=150,
    breakpoint_threshold_type="percentile"
)
sem_chunks = semantic_splitter.create_documents([d.page_content for d in rec_chunks])
sem_chunks = [doc for doc in sem_chunks if 200 < len(doc.page_content) < 1200]

# STEP 4: Prompt/Response Pair Creation
import json
import random

prompt_templates = [
    "Summarize this guidance:\n\n{}",
    "What does this mean in practice?\n\n{}",
    "Explain the following in simple terms:\n\n{}",
    "As a cybersecurity professional, how should I interpret this?\n\n{}"
]

def make_prompt_response_pairs(chunks):
    prompt_response_pairs = []
    for chunk in sem_chunks:
        prompt = f"Explain the following in simple terms:\n\n{chunk}"
        response = chunk.strip()
        prompt_response_pairs.append({"input": prompt, "output": response})
    return prompt_response_pairs


def make_diverse_pairs(chunks):
    prompt_response_pairs = []
    for chunk in sem_chunks:
        template = random.choice(prompt_templates)
        prompt = template.format(chunk.strip())
        response = chunk.strip()
        prompt_response_pairs.append({"input": prompt, "output": response})
    return prompt_response_pairs


# Save as JSONL
with open("nist_full_dataset.jsonl", "w") as f:
    for entry in prompt_response_pairs:
        json.dump(entry, f)
        f.write("\n")

print(f" Saved {len(prompt_response_pairs)} prompt/response pairs.")

# STEP 5: Load Dataset
from datasets import load_dataset
dataset = load_dataset("json", data_files="nist_full_dataset.jsonl", split="train")
print("Example:\n", dataset[0])

# STEP 6: Load 4-bit Model with PEFT
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_name = "fdtn-ai/Foundation-Sec-8B"

bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

base_model = prepare_model_for_kbit_training(base_model, use_gradient_checkpointing=True)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# STEP 7: Tokenize Dataset
def format(example):
    full_prompt = f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}"
    tokens = tokenizer(full_prompt, padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(format)

# STEP 8: Fine-Tune (Safe for 14 GB GPU)
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="fine_tuned_fs8b_nist",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

# STEP 9: Save Model
model.save_pretrained("/content/fs8b-nist-finetuned")
tokenizer.save_pretrained("/content/fs8b-nist-finetuned")
!cp -r /content/fs8b-nist-finetuned /content/drive/MyDrive/
print(" Model saved to Drive.")

✅ Extracted 761 paragraphs.
✅ Saved 818 prompt/response pairs.


Generating train split: 0 examples [00:00, ? examples/s]

Example:
 {'input': 'Explain the following excerpt from NIST CSF 2.0:\n\nNational Institute of Standards and Technology \nThis publication is available free of charge from: https://doi.org/10.6028/NIST.CSWP.29 \nFebruary 26, 2024  \nThe NIST Cybersecurity \nFramework (CSF) 2.0', 'output': 'National Institute of Standards and Technology \nThis publication is available free of charge from: https://doi.org/10.6028/NIST.CSWP.29 \nFebruary 26, 2024  \nThe NIST Cybersecurity \nFramework (CSF) 2.0\n\n(This is a direct explanation from NIST CSF 2.0. Expand as needed.)'}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/tmp/ipython-input-2133894255.py:117: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*arg

Step,Training Loss
10,8.710800
20,0.668600
30,0.404600
40,0.194700
50,0.193500
60,0.142300
70,0.184500
80,0.159100
90,0.130600
100,0.130300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

✅ Model saved to Drive.


In [ ]:
!pip install bitsandbytes accelerate transformers peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# === Configs ===
base_model = "fdtn-ai/Foundation-Sec-8B"
peft_model_path = "/content/drive/MyDrive/fs8b-nist-finetuned"

# === Correct Quantization Config ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# === Load tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(base_model)

# === Load quantized base model ===
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)

# === Load LoRA adapters + merge ===
model = PeftModel.from_pretrained(model, peft_model_path)
model = model.merge_and_unload()
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128384, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [ ]:
prompt = """What are the core functions of NIST CSF 2.0?"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.1,  # discourages repeating tokens
    eos_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[ROLE: Security Architect]
What are the core functions of NIST CSF 2.0?
NIST CSF 2.0 is a framework designed to help organizations manage and reduce cybersecurity risks. It provides guidelines, best practices, and standards for managing cybersecurity risk across an organization’s entire enterprise. The framework consists of five main functions: Identify, Protect, Detect, Respond, and Recover. Each function has its own set of activities that can be implemented in order to achieve desired outcomes related to each specific area. For example:
- Identify – This involves understanding what assets need protection from potential threats; this includes both internal and external factors such as personnel policies or physical security measures.
- Protect – Here you will find guidance on how best protect your systems through technical controls like encryption technologies or authentication methods used by users when accessing company resources online.
- Detect – In this section there are recommen